In [1]:
from PIL import Image
import pytesseract
import os
from tempfile import TemporaryDirectory
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from pdf2image import convert_from_path
from PyPDF2 import PdfWriter, PdfReader
import os
import asyncio
import urllib3.request
import time
from PyPDF2 import errors

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
DIR = os.getenv('KEA_BASE_DIR')

In [4]:
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "eager"

options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}

options.add_experimental_option('detach', True)
options.add_experimental_option("prefs", prefs)
options.add_argument('--headless')

s = Service(f"{DIR}/chromedriver/chromedriver.exe")

In [6]:
def has_pagination(href):
    return href and ("javascript:__doPostBack('ctl00$ContentPlaceHolder1$GridView1" in href)

def has_polling_station(tag):
    is_correct_tag = False
    try:
        mask1 = r"CodeCaputer1.aspx?field1=.%2fKANNADA%2fMR%2f" in tag.find('a')['href']
        mask2 = bool(re.search("background-color",tag['style']))
        mask3 = tag.name=='tr'
        is_correct_tag = mask1 and mask2 and mask3
    except:
        pass
    return is_correct_tag

page_dict = {}

driver = webdriver.Chrome(options=options, service=s, desired_capabilities=caps)

for i in range(1,225):
    if i%50==0:
        driver.close()
        time.sleep(120)
        driver = webdriver.Chrome(options=options, service=s, desired_capabilities=caps)
    hit_url = True
    get_url = 0
    while hit_url and get_url<2:
        try:
            driver.get(f'https://ceo.karnataka.gov.in/Elections_rolls_2023/Part_List.aspx?ACNO={i}')
        except:
            print(f"Couldn't hit url with const number {i}")
            get_url += 1
            time.sleep(90)
        else:
            hit_url = False
    if hit_url: continue
    soup = BeautifulSoup(driver.page_source,'html.parser')
    temp_list = soup.find_all('a',href=has_pagination,style="color:White;")
    driver.execute_script(temp_list[-1]['href'])      
    page_counter = len(temp_list)
    while True:
        time.sleep(2.5)
        driver.execute_script(f"javascript:__doPostBack('ctl00$ContentPlaceHolder1$GridView1','Page${page_counter}')")
        soup = BeautifulSoup(driver.page_source,'html.parser')
        if not soup.find('h1',string='Server Error'):
            try:
                temp_list=soup.find_all(has_polling_station)
                max_stations = int(temp_list[-1].find_all('td')[1].string.strip())
            except:
                print(f"Error while parsing HTML for const number {i} and page number {page_counter}")
                break
        else:
            break
        page_counter += 1
    page_dict.update({i:max_stations})
    print(f"constituency {i} has {max_stations} pages")
    time.sleep(3.5)
        
driver.close()

constituency 1 has 246 pages
constituency 2 has 244 pages
constituency 3 has 260 pages
Error while parsing HTML for const number 4 and page number 13
constituency 4 has 231 pages
constituency 5 has 217 pages
constituency 6 has 228 pages
constituency 7 has 224 pages
constituency 8 has 281 pages
constituency 9 has 288 pages
constituency 10 has 235 pages
constituency 11 has 249 pages
constituency 12 has 251 pages
Error while parsing HTML for const number 13 and page number 16
constituency 13 has 292 pages
constituency 14 has 255 pages
constituency 15 has 230 pages
constituency 16 has 224 pages
constituency 17 has 232 pages
constituency 18 has 247 pages
constituency 19 has 212 pages
constituency 20 has 235 pages
constituency 21 has 232 pages
constituency 22 has 263 pages
constituency 23 has 260 pages
constituency 24 has 263 pages
constituency 25 has 254 pages
constituency 26 has 241 pages
constituency 27 has 252 pages
constituency 28 has 232 pages
constituency 29 has 243 pages
constituency

In [7]:
sum(list(page_dict.values())) # Total number of polling stations in Karnataka

58282

In [38]:
with open('new_links.txt','w',encoding='utf-8') as file:
    for key in list(page_dict.keys()):
        for page in range(1,page_dict[key]+1):
            A = key
            AC = f"00{key}" if A<10 else f"0{key}" if A<100 else f"{key}"
            link = f"https://ceo.karnataka.gov.in/Elections_rolls_2023/KANNADA/MR/AC{AC}/S10A{A}P{page}.pdf"
            file.write(link+'\n')

*========================================================================================================================*